In [53]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [43]:
df = pd.read_csv('train.csv')
df.head(10)

,rally,ball_round,time,frame_num,roundscore_A,roundscore_B,player,type,aroundhead,backhand,landing_height,landing_area,landing_x,landing_y,lose_reason,getpoint_player,player_location_area,player_location_x,player_location_y,opponent_location_area,opponent_location_x,opponent_location_y,set,match_id,rally_id,rally_length
0,1,1,00:06:00,10827,1,0,0,short service,0.0,1,2.0,7,0.773171,0.519687,NaN,NaN,8,170.8,308.95,8,236.7,675.21,1,1,0,8
1,1,2,00:06:01,10849,1,0,1,net shot,0.0,0,2.0,7,0.086585,-0.237240,NaN,NaN,8,208.7,594.37,8,176.8,318.46,1,1,0,8
2,1,3,00:06:02,10873,1,0,0,lob,0.0,0,1.0,3,1.402439,1.638490,NaN,NaN,7,174.2,378.28,8,180.1,622.54,1,1,0,8
3,1,4,00:06:03,10917,1,0,1,clear,0.0,0,1.0,4,1.358537,-1.434167,NaN,NaN,3,257.9,809.08,8,179.3,306.43,1,1,0,8
4,1,5,00:06:04,10943,1,0,0,drop,0.0,1,2.0,1,1.487805,0.359844,NaN,NaN,6,273.5,268.22,8,233.2,738.15,1,1,0,8
5,1,6,00:06:05,10966,1,0,1,net shot,0.0,0,2.0,2,1.297561,-0.220104,NaN,NaN,1,259.8,573.61,8,240.6,263.58,1,1,0,8
6,1,7,00:06:06,10994,1,0,0,lob,0.0,1,1.0,9,-0.787805,1.659635,NaN,NaN,2,255.6,384.09,8,211.5,604.38,1,1,0,8
7,1,8,00:06:08,11041,1,0,1,drop,0.0,0,2.0,10,1.601220,-0.504062,out,0.0,9,128.9,789.40,8,154.2,292.12,1,1,0,8
8,8,1,00:09:20,16805,2,6,1,short service,0.0,1,2.0,7,-0.280488,-0.495677,NaN,NaN,8,186.0,630.55,8,132.6,284.99,1,1,7,5
9,8,2,00:09:21,16830,2,6,0,push/rush,0.0,1,1.0,3,1.204878,1.544219,NaN,NaN,7,125.7,391.98,8,166.2,633.83,1,1,7,5


In [138]:
typemap = {}
for idx, typ in enumerate(np.unique(df['type'])):
    typemap[typ] = idx
typemap        

{'clear': 0,
 'defensive shot': 1,
 'drive': 2,
 'drop': 3,
 'lob': 4,
 'long service': 5,
 'net shot': 6,
 'push/rush': 7,
 'short service': 8,
 'smash': 9}

In [125]:
class dataset(Dataset):
    def __init__(self, df):
        typemap = {}
        for idx, typ in enumerate(np.unique(df['type'])):
            typemap[typ] = idx

        self.data = pd.DataFrame()
        self.data['type'] = df['type'].apply(lambda x: typemap[x])
        self.data['landing_x'] = df['landing_x']
        self.data['landing_y'] = df['landing_y']
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.tensor([self.data['type'][idx]], dtype=torch.float), torch.tensor([self.data['landing_x'][idx], self.data['landing_y'][idx]])

In [126]:
class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.block = nn.Sequential(*[
            nn.Linear(1, 16),
            nn.Linear(16, 32),
            nn.Linear(32, 16),
            nn.Linear(16, 2),
        ])
    
    def forward(self, x):
        return self.block(x)

In [133]:
lr = 3e-4
n_iters = 500

In [128]:
df = pd.read_csv('train.csv')
len_df = len(df)
train_df = df[:int(len_df * 80 / 100)]
test_df = df[int(len_df * 80 / 100):]

train = dataset(train_df)
test = dataset(test_df)
train_loader = DataLoader(train, batch_size=64, shuffle=True)
test_loader = DataLoader(test, batch_size=64, shuffle=True)

In [129]:
m = model()
optimizer = optim.AdamW(m.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [130]:
train[0]

(tensor([8.]), tensor([0.7732, 0.5197], dtype=torch.float64))

In [134]:
for i in range(n_iters):
    for x, y in train_loader:
        logits = m(x)
        loss = criterion(logits, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if i % 100 == 0 or i == n_iters:
        print(f'{i = }, loss = {loss.item()}')

i = 0, loss = 0.39240990465882125
i = 100, loss = 0.14253572005121493
i = 200, loss = -0.4347799233607771
i = 300, loss = -0.60830965435047
i = 400, loss = 0.6933463345105149


In [140]:
m(torch.tensor([6.]))

tensor([-0.3889,  0.4075], grad_fn=<AddBackward0>)